In [1]:
import cv2 as cv
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [6]:
labels = {}
with open("./find_phone/labels.txt") as f:
    for line in f:
        (key, val1, val2) = line.split()
        labels[key] = [float(val1), float(val2)]

In [86]:
def center_of_rectangle(rect):
    x = (rect[0] + rect[2])/2
    y = (rect[1] + rect[3])/2
    return (x/490, y/326)

In [87]:
for key in labels:
    image = cv.imread("./find_phone/" + key )
    y,x,_ = image.shape
    centerx, centery = int(float(labels[key][0])*x), int(float(labels[key][1])*y)
    # cv.rectangle(image, mask[0], mask[1], (0,0,255), 2)
    ROI_points = np.array([[(centerx-30, centery-30),(centerx+30,centery-30),(centerx+30, centery+30),(centerx-30, centery+30)]], dtype=np.int32)
    masked_image = np.zeros(image.shape[:2], dtype=np.uint8)
    mask = cv.fillPoly(masked_image,ROI_points,255)
    ## Apply mask to edges image
    masked_image = cv.bitwise_and(image,image, mask = mask)
    
    
    l,a,b = cv.split(cv.cvtColor(masked_image,cv.COLOR_BGR2LAB))
    clahe = cv.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    adpt_hist = clahe.apply(l)
    corrected_image = cv.cvtColor((cv.merge((adpt_hist,a,b))),cv.COLOR_LAB2BGR)
    
    blurred = cv.GaussianBlur(corrected_image,(3,3),0)
    # gray = cv.cvtColor(blurred[:,:,-1],cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(blurred[:,:,0], 150, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    canny = cv.Canny(thresh,100,200)
    contours, hierarchy = cv.findContours(image=thresh, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_SIMPLE)
    try:
        second_largerst = np.argsort([cv.contourArea(x)for x in contours])[-2]
    except:
        second_largerst = 0
    # cv.drawContours(image, contours[second_largerst], -1, (0,255,0), 3)
    # print(labels[key],cv.mom)
    
    x,y,w,h = cv.boundingRect(contours[second_largerst])
    cv.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    
    print(labels[key], center_of_rectangle([x,y,x+w,y+h]))
    
    cv.namedWindow("image", cv.WINDOW_NORMAL)
    cv.imshow("image",thresh)
    cv.namedWindow("image2", cv.WINDOW_NORMAL)
    cv.imshow("image2",canny)
    cv.namedWindow("image3", cv.WINDOW_NORMAL)
    cv.imshow("image3",image)
    
    # print(labels[key],contours[-1])
    if cv.waitKey(0) & 0xFF == ord('s'):
            cv.destroyAllWindows()
            break
    # print(key, labels[key])
    # break
cv.waitKey(1)
cv.destroyAllWindows()

[0.2388, 0.6012] (0.24081632653061225, 0.6058282208588958)
[0.2551, 0.3129] (0.2612244897959184, 0.31901840490797545)
[0.7122, 0.7117] (0.7030612244897959, 0.7392638036809815)
[0.5286, 0.4877] (0.5418367346938775, 0.49693251533742333)
[0.7714, 0.3252] (0.7714285714285715, 0.32515337423312884)
[0.1122, 0.3466] (0.11428571428571428, 0.3512269938650307)
[0.8449, 0.1564] (0.8530612244897959, 0.17638036809815952)
[0.4837, 0.4509] (0.49489795918367346, 0.4708588957055215)
[0.2612, 0.8037] (0.25918367346938775, 0.803680981595092)
[0.6551, 0.3466] (0.6591836734693878, 0.348159509202454)
[0.1878, 0.1104] (0.18979591836734694, 0.11503067484662577)
[0.7837, 0.6503] (0.7836734693877551, 0.6503067484662577)
[0.602, 0.1135] (0.6051020408163266, 0.11349693251533742)
[0.149, 0.6442] (0.15, 0.6457055214723927)
[0.8061, 0.2699] (0.810204081632653, 0.27607361963190186)
[0.4082, 0.7791] (0.4020408163265306, 0.7684049079754601)
[0.4327, 0.8405] (0.4346938775510204, 0.8450920245398773)
[0.2286, 0.8712] (0.2

In [88]:
ROI_points

array([[[200, 143],
        [260, 143],
        [260, 203],
        [200, 203]]])

In [65]:
np.argsort([cv.contourArea(x)for x in contours])[-2]

array([3, 2, 1, 4, 0], dtype=int64)